In [314]:
%reset

In [315]:
import os ########################################
import numpy as np ###############################
import scipy.stats as stats ######################
import math ######################################
import matplotlib.pyplot as plt###################
from sklearn.metrics import mean_squared_error####

In [316]:
class reconstruct(object):
    
    def __init__(self, edge, bin_data, GMM, label, Num_of_bins, x, y, z=0):
        self.edge = edge
        self.bin_data = bin_data
        self.GMM = GMM
        self.label = label
        self.x = x
        self.y = y
        self.z = z
        self.Num_of_bins = Num_of_bins
        if z != 0:
            self.reconstructed_data = np.full((self.x, self.y, self.z, self.Num_of_bins), 0)
        else:
            self.reconstructed_data = np.full((self.x, self.y, self.Num_of_bins), 0)
    
    def reconstruct(self, time): #bin: [index, prob, Gmm label, Num of component]
        if self.z!=0:
            for x in range(self.x):
                for y in range(self.y):
                    for z in range(self.z):
                        self.reconstructed_data[x,y,z] = self.grid_reconstruct(self.bin_data[x,y,z], time)
        else:
            for x in range(self.x):
                for y in range(self.y):
                    self.reconstructed_data[x,y] = self.grid_reconstruct(self.bin_data[x,y], time)

    def grid_reconstruct(self, grid, time_step):
        numOfbin = len(grid)
        f = np.zeros(numOfbin)
        p = np.zeros(numOfbin)
        bin_count=0
        for bin in grid:
            if(bin[0] == -1): 
                f[bin_count] = 0
                p[bin_count] = 0
            else:
                GMM_prob = 0
                bin_index = bin[0]
                bar_prob = bin[1]
                Gmm_label = int(bin[2])
                start = int(self.label[Gmm_label])
                n_comp = int(bin[3])
                end = int(start+n_comp)
                # print('start: ', start, 'end: ',end)
                weights = self.GMM[0][start:end]
                means = self.GMM[1][start:end]
                covars = self.GMM[2][start:end]
                for n_components in range(n_comp):
                    GMM_prob += weights[n_components] * (np.exp((-((time_step-means[n_components])**2))/(2*(np.sqrt(covars[n_components])**2))) / np.sqrt(2*np.pi*(np.sqrt(covars[n_components])**2)))
                f[bin_count] = GMM_prob
                p[bin_count] = bar_prob
                start += int(n_comp)
            bin_count += 1
        postProb = f * p
        postProbSum = np.sum(postProb)
        postProb = postProb / postProbSum
        return postProb
                                       
    
    def _compute_cdf(self, pdf, edge, x):
        allcdf = np.cumsum(pdf)
        allcdf = np.hstack([0.0, allcdf])
        cdf = np.interp(x, edge, allcdf, left=0, right=0)
        return cdf
           
    
    def compute_isovalue(self, reconst_data, iso_value):
        hist_edge = self.edge
        
        if self.z!=0:
            cdf = np.zeros((self.x, self.y, self.z))
            for i in range(self.x):
                for j in range(self.y):
                    for k in range(self.z):
                        cdf[i,j,k] = self._compute_cdf(reconst_data[i,j,k], hist_edge, iso_value)
            cdf_1_c = 1 - cdf
            pCrossing = np.zeros((self.x, self.y, self.z))
            for l in range(self.x):
                for m in range(self.y):
                    for n in range(self.z):
                        if(l == self.x-1):
                            pCrossing[l, m, n] = pCrossing[l-1, m, n]
                        elif(m == self.y-1):
                            pCrossing[l, m, n] = pCrossing[l, m-1, n]
                        elif(n == self.z-1):
                            pCrossing[l, m, n] = pCrossing[l, m, n-1]   
                        else:
                            prob = cdf[l, m, n] * cdf[l+1, m, n] * cdf[l, m+1, n] * cdf[l, m, n+1] * cdf[l+1, m+1, n] * cdf[l+1, m, n+1] * cdf[l, m+1, n+1] * cdf[l+1, m+1, n+1]
                            prob2 = cdf_1_c[l, m, n] * cdf_1_c[l+1, m, n] * cdf_1_c[l, m+1, n] * cdf_1_c[l, m, n+1] * cdf_1_c[l+1, m+1, n] * cdf_1_c[l+1, m, n+1] * cdf_1_c[l, m+1, n+1] * cdf_1_c[l+1, m+1, n+1]
                            pCrossing[l, m, n] = 1 - prob - prob2
        else:    
            cdf = np.zeros((self.x, self.y))
            for i in range(self.x):
                for j in range(self.y):
                    cdf[i,j] = self._compute_cdf(reconst_data[i,j], hist_edge, iso_value)
            cdf_1_c = 1 - cdf
            pCrossing = np.zeros((self.x, self.y))
            for k in range(self.x):
                for l in range(self.y):
                    if(l == self.y-1):
                        pCrossing[k, l] = pCrossing[k, l-1]
                    elif(k == self.x-1):
                        pCrossing[k, l] = pCrossing[k-1, l]
                    else:
                        prob = cdf[k, l] * cdf[k, l+1] * cdf[k+1, l] * cdf[k+1, l+1]
                        prob2 = cdf_1_c[k, l] * cdf_1_c[k, l+1] * cdf_1_c[k+1, l] * cdf_1_c[k+1, l+1]
                        pCrossing[k, l] = 1 - prob - prob2
        return pCrossing
        
    
    def all_reconstruct(self, time): #bin: [index, prob, Gmm label, Num of component]
        if self.z!=0:
            reconstructed_data = np.zeros((self.x, self.y, self.z, self.Num_of_bins, time))
            for x in range(self.x):
                for y in range(self.y):
                    for z in range(self.z):
                        reconstructed_data[x,y,z] = self._all_times_reconstruct(self.bin_data[x,y,z], time)
        else:
            reconstructed_data = np.zeros((self.x, self.y, self.Num_of_bins, time))
            for x in range(self.x):
                for y in range(self.y):
                    reconstructed_data[x,y] = self._all_times_reconstruct(self.bin_data[x,y], time)
        return reconstructed_data
    
    def new_all_reconstruct(self, time):
        if self.z != 0 :
            reconstruct_bin = np.full((self.x, self.y, self.z, self.Num_of_bins, 4), -1.00)
            reconstructed_data = np.zeros((self.x, self.y, self.z, self.Num_of_bins, time))
            for bin in self.bin_data:
                grid_index = bin[0]
                bin_index = int(bin[1])
                x_location = int(grid_index//(self.y*self.z))
                y_location = int((grid_index%(self.y*self.z))//self.z)
                z_location = int((grid_index%(self.y*self.z))%self.z)
                reconstruct_bin[x_location, y_location, z_location, bin_index] = bin[1:]
            for x in range(self.x):
                for y in range(self.y):
                    for z in range(self.z):
                        reconstructed_data[x,y,z] = self._all_times_reconstruct(reconstruct_bin[x,y,z], time)
        else:
            reconstruct_bin = np.full((self.x, self.y, self.Num_of_bins, 4), -1.00)
            reconstructed_data = np.zeros((self.x, self.y, self.Num_of_bins, time))
            for bin in self.bin_data:
                grid_index = int(bin[0])
                bin_index = int(bin[1])
                x_location = int(grid_index//self.y)
                y_location = int(grid_index%self.y)
                reconstruct_bin[x_location, y_location, bin_index] = bin[1:]
            for x in range(self.x):
                for y in range(self.y):
                    reconstructed_data[x,y] = self._all_times_reconstruct(reconstruct_bin[x,y], time)
        return reconstructed_data
            
    
    def _all_times_reconstruct(self, grid, time):
        all_time_stpes = np.arange(time)
        numOfbin = len(grid)
        postProb = np.zeros((numOfbin,len(all_time_stpes)))
        bin_count=0
        SumpostProb = np.zeros(len(all_time_stpes))
        for bin in grid:
            if(bin[0] == -1):
                bin_count +=1
            else:
                GMM_prob = np.zeros(len(all_time_stpes))
                bin_index = bin[0]
                bar_prob = bin[1]
                Gmm_label = int(bin[2])
                start = int(self.label[Gmm_label])
                n_comp = int(bin[3])
                end = int(start+n_comp)
                weights = self.GMM[0][start:end]
                means = self.GMM[1][start:end]
                covars = self.GMM[2][start:end]
                for n_components in range(n_comp):
                        GMM_prob += weights[n_components] * (np.exp((-((all_time_stpes-means[n_components])**2))/(2*(np.sqrt(covars[n_components])**2))) / 
                                                             np.sqrt(2*np.pi*(np.sqrt(covars[n_components])**2)))
                tempPostProb = GMM_prob * bar_prob 
                SumpostProb += tempPostProb 
                postProb[bin_count] = tempPostProb
                start += int(n_comp)
                bin_count += 1
        postProb = postProb/SumpostProb
        return postProb


In [361]:
def interpolated_hist(data, numOfTimeStep, numOfensemble, interval):
    data_size = data.shape
    min_of_data = data.min()    
    max_of_data = data.max()
    NumOfBinsL = 0
    NumOfBinsR = 0
    x = data_size[0]
    y = data_size[1]
    z = data_size[2]
    if(len(data_size)>3):
        inter_hist_prob = np.zeros((x,y,z,numOfTimeStep,128))
    else:
        inter_hist_prob = np.zeros((x,y,numOfTimeStep,128))
    # print('interp_hist_prob shape', inter_hist_prob.shape)
    selected_time = np.arange(0, numOfTimeStep, interval)
    selected_time = selected_time-1
    selected_time[0] = 0
    endtime = selected_time[-1]+interval
    if endtime >= numOfTimeStep:
        endtime = numOfTimeStep-1
    selected_time = np.append(selected_time,endtime)
    print(selected_time)
    for select_index in range(len(selected_time)-1):
        left_time = selected_time[select_index]
        right_time = selected_time[select_index+1]
        # print('left: ', left_time,' right: ',right_time)
        if(len(data_size)>3):
            for i in range(x):
                for j in range(y):
                    for k in range(z):
                        reconstruct_time = np.arange(left_time,right_time)
                        rawdataL = data[i, j, k, (left_time*numOfensemble):(left_time*numOfensemble)+numOfensemble]
                        rawdataR = data[i, j, k, (right_time*numOfensemble):(right_time*numOfensemble)+numOfensemble]
                        histL = np.histogram(rawdataL, bins=128, range=(min_of_data, max_of_data))[0]
                        histR = np.histogram(rawdataR, bins=128, range=(min_of_data, max_of_data))[0]
                        NumOfBinsL += np.count_nonzero(histL)
                        # print('L: ',NumOfBinsL)
                        if right_time == selected_time[-1]:
                            NumOfBinsR  += np.count_nonzero(histR)
                            # print('R: ',NumOfBinsR)
                        for time_index in reconstruct_time:
                            interhist = histL + (((histR-histL)/interval) * (time_index-left_time))
                            inter_hist_prob[i,j,k,time_index] = interhist/np.sum(interhist)
            # print('left time:',left_time,' num_of_bins: ', (NumOfBinsL+NumOfBinsR))                    
        else:
            for i in range(x):
                for j in range(y):
                    reconstruct_time = np.arange(left_time,right_time)
                    rawdataL = data[i, j, (left_time*numOfensemble):(left_time*numOfensemble)+numOfensemble]
                    rawdataR = data[i, j, (right_time*numOfensemble):(right_time*numOfensemble)+numOfensemble]
                    histL = np.histogram(rawdataL, bins=128, range=(min_of_data, max_of_data))[0]
                    histR = np.histogram(rawdataR, bins=128, range=(min_of_data, max_of_data))[0]
                    NumOfBinsL += np.count_nonzero(histL)
                    if right_time == selected_time[-1]:
                        NumOfBinsR  += np.count_nonzero(histR)
                    for time_index in reconstruct_time:
                        interhist = histL + (((histR-histL)/interval) * (time_index-left_time))
                        inter_hist_prob[i,j,time_index] = interhist/np.sum(interhist)
    storage = (((NumOfBinsL+NumOfBinsR)*4*32)/8)/1000000  
    print('storage: ', storage,'(MB)')
    return inter_hist_prob
    

In [318]:
def compute_error(reconstruction, raw):
    data_size = reconstruction.shape
    if (len(reconstruction.shape) >3):
        data_x = data_size[0]
        data_y = data_size[1]
        data_z = data_size[2] 
        rmse = np.sum((np.sum((reconstruction - raw)**2, axis=3)/128)**(1/2))
        rmse = rmse/(data_x*data_y*data_z)
    else:
        data_x = data_size[0]
        data_y = data_size[1]
        rmse = np.sum((np.sum((reconstruction - raw)**2, axis=2)/128)**(1/2))
        rmse = rmse/(data_x*data_y)
    return rmse               

In [319]:
def compute_3Disovalue(hist_dist, isovalue):
    pcrossing = np.zeros((32,32,32))
    for i in range(32):
        for j in range(32):
            for k in range(32):
                if(j == 31):
                    pcrossing[i, j, k] = pcrossing[i, j-1, k]
                elif(i == 31):
                    pcrossing[i, j, k] = pcrossing[i-1, j, k]
                elif(k == 31):
                    pcrossing[i, j, k] = pcrossing[i, j, k-1]
                else:
                    prob = hist_dist[i, j, k].cdf(isovalue)*hist_dist[i,j+1,k].cdf(isovalue)*hist_dist[i+1,j,k].cdf(isovalue)*hist_dist[i,j,k+1].cdf(isovalue)*hist_dist[i+1,j+1,k].cdf(isovalue)*hist_dist[i+1,j,k+1].cdf(isovalue)*hist_dist[i,j+1,k+1].cdf(isovalue)*hist_dist[i+1,j+1,k+1].cdf(isovalue)
                    prob2 = (1-hist_dist[i, j,k].cdf(isovalue))*(1-hist_dist[i, j+1,k].cdf(isovalue))*(1-hist_dist[i+1, j,k].cdf(isovalue))*(1-hist_dist[i, j, k+1].cdf(isovalue))*(1-hist_dist[i+1, j+1, k].cdf(isovalue))*(1-hist_dist[i+1, j, k+1].cdf(isovalue))*(1-hist_dist[i, j+1, k+1].cdf(isovalue))*(1-hist_dist[i+1, j+1, k+1].cdf(isovalue))
                    pcrossing[i, j, k] = 1 - prob - prob2
    return pcrossing 

In [320]:
def compute_isovalue(hist_dist, isovalue):
    pcrossing = np.zeros((160,320))
    for i in range(160):
        for j in range(320):
            if(j == 159):
                pcrossing[i, j] = pcrossing[i, j-1]
            elif(i == 319):
                pcrossing[i, j] = pcrossing[i-1, j]
            else:
                prob = hist_dist[i, j].cdf(isovalue)*hist_dist[i,j+1].cdf(isovalue)*hist_dist[i+1,j].cdf(isovalue)*hist_dist[i+1,j+1].cdf(isovalue)
                prob2 = (1-hist_dist[i, j].cdf(isovalue))*(1-hist_dist[i, j+1].cdf(isovalue))*(1-hist_dist[i+1, j].cdf(isovalue))*(1-hist_dist[i+1, j+1].cdf(isovalue))
                pcrossing[i, j] = 1 - prob - prob2
    return pcrossing    

In [321]:
def compute_interp_isovalue(interp_data ,edge, isovalue):
    if len(interp_data.shape) > 3:
        x = interp_data.shape[0]
        y = interp_data.shape[1]
        z = interp_data.shape[2]
        cdf = np.zeros((x,y,z))
        for i in range(x):
            for j in range(y):
                for k in range(z):
                    allcdf = np.cumsum(interp_data[i,j,k])
                    allcdf = np.hstack([0.0, allcdf])
                    cdf[i,j,k] = np.interp(isovalue, edge, allcdf, left=0, right=0)
        cdf_1_c = 1 - cdf
        pCrossing = np.zeros((x,y,z))
        for l in range(x):
            for m in range(y):
                for n in range(z):
                    if(l == x-1):
                        pCrossing[l,m,n] = pCrossing[l-1,m,n]
                    elif(m == y-1):
                        pCrossing[l,m,n] = pCrossing[l,m-1,n]
                    elif(n == z-1):
                        pCrossing[l,m,n] = pCrossing[l,m,n-1]
                    else:
                        prob = cdf[l, m, n] * cdf[l+1, m, n] * cdf[l, m+1, n] * cdf[l, m, n+1] * cdf[l+1, m+1, n] * cdf[l+1, m, n+1] * cdf[l, m+1, n+1] * cdf[l+1, m+1, n+1]
                        prob2 = cdf_1_c[l, m, n] * cdf_1_c[l+1, m, n] * cdf_1_c[l, m+1, n] * cdf_1_c[l, m, n+1] * cdf_1_c[l+1, m+1, n] * cdf_1_c[l+1, m, n+1] * cdf_1_c[l, m+1, n+1] * cdf_1_c[l+1, m+1, n+1]
                        pCrossing[l, m, n] = 1 - prob - prob2
    else:
        x = interp_data.shape[0]
        y = interp_data.shape[1]
        cdf = np.zeros((x,y))
        for i in range(x):
            for j in range(y):
                allcdf = np.cumsum(interp_data[i,j])
                allcdf = np.hstack([0.0, allcdf])
                cdf[i,j] = np.interp(isovalue, edge, allcdf, left=0, right=0)
        cdf_1_c = 1 - cdf
        pCrossing = np.zeros((x,y))
        for k in range(x):
            for l in range(y):
                if(l == y-1):
                    pCrossing[k,l] = pCrossing[k,l-1]
                elif(k == x-1):
                    pCrossing[k,l] = pCrossing[k-1,l]
                else:
                    prob = cdf[k, l] * cdf[k, l+1] * cdf[k+1, l] * cdf[k+1, l+1]
                    prob2 = cdf_1_c[k, l] * cdf_1_c[k, l+1] * cdf_1_c[k+1, l] * cdf_1_c[k+1, l+1]
                    pCrossing[k, l] = 1 - prob - prob2
    return pCrossing            
        

In [322]:
def reconstruct_raw_bin(bin_data, x, y, z, TotalTimesteps, NumOfBins, NumOfEnsemble):
    if(z != 0):
       Reconted_data = np.zeros((TotalTimesteps, x, y, z, NumOfBins))
       for bin in bin_data:
           timestep = int(bin[0])
           x_location = int(bin[1]//(y*z))
           y_location = int((bin[1]%(y*z))//z)
           z_location = int((bin[1]%(y*z))%z)
           bin_index = int(bin[2])
           bin_prob = bin[3]/NumOfEnsemble
           Reconted_data[timestep, x_location, y_location, z_location, bin_index] = bin_prob
    else:
        Reconted_data = np.zeros((int(bin_data[:,0].max()+1), x, y, NumOfBins))
        for bin in bin_data:
            timestep = int(bin[0])
            x_location = int(bin[1]//y)
            y_location = int(bin[1]%y)
            if(y_location>=128):
                bin_index = 127
            bin_index = int(bin[2])
            if(bin_index>=128):
                bin_index = 127
            bin_prob = bin[3]/NumOfEnsemble
            Reconted_data[timestep, x_location, y_location, bin_index] = bin_prob
    return Reconted_data

In [323]:
def quick_create_reconstruct(floder_path, num_of_bins, start_time, end_time, x, y, z=0):
    # bin_data_name = 'bin_data_x160_y320_NumOfbBins128_time_range_{start_time}_{end_time}.bin'.format(start_time=start_time, end_time=end_time)
    # bin_data_name = 'bin_data_x32_y32_z32_NumOfBins128_RangeOfTimestime_range_{start_time}_{end_time}.bin'.format(start_time=start_time, end_time=end_time)
    bin_data_name = 'bin_data_x25_y20_z120_NumOfBins128_RangeOfTimestime_range_{start_time}_{end_time}.bin'.format(start_time=start_time, end_time=end_time)
    edge_name = 'edge_time_range_{start_time}_{end_time}.bin'.format(start_time=start_time, end_time=end_time)
    model_name = 'Gmm_time_range_{start_time}_{end_time}.bin'.format(start_time=start_time, end_time=end_time)
    label_name = 'label_time_range_{start_time}_{end_time}.bin'.format(start_time=start_time, end_time=end_time)
    bin_data_path = os.path.join(floder_path, bin_data_name)
    edge_path = os.path.join(floder_path, edge_name)
    model_path = os.path.join(floder_path, model_name)
    
    label_path = os.path.join(floder_path, label_name)
    bin_data = np.fromfile(bin_data_path, dtype=np.float32)
    bin_data = bin_data.reshape((len(bin_data)//5, 5))
    edge = np.fromfile(edge_path, dtype=np.float32)
    model = np.fromfile(model_path, dtype=np.float32)
    model = model.reshape((3, len(model)//3))
    label = np.fromfile(label_path, dtype=np.float32)
    # print('bin data name: ', bin_data_name)
    # print('edge name: ', edge_name)
    # print('model name: ', model_name)
    # print('label name: ', label_name)
    
    return reconstruct(edge, bin_data, model, label, num_of_bins, x, y, z)    

In [324]:
def compute_file_error(floder_path, raw_hist_data, timesteps, cons_time, num_of_bins, x, y, z=0):
    error = np.zeros(timesteps)
    for s_time in range(0, timesteps, cons_time):
        end_time = s_time+cons_time
        # print('start: ', s_time, ' end: ', end_time)
        reconstruction = quick_create_reconstruct(floder_path, num_of_bins, s_time, end_time, x, y, z)
        reconstruction_data = reconstruction.new_all_reconstruct(cons_time)
        if (z == 0):
            for timestep in range(cons_time):
                error[s_time+timestep] = compute_error(reconstruction_data[:,:,:,timestep], raw_hist_data[s_time+timestep])
        else:
            for timestep in range(cons_time):
                error[s_time+timestep] = compute_error(reconstruction_data[:,:,:,:,timestep], raw_hist_data[s_time+timestep])
    error.astype('float32').tofile(floder_path+'\\timesteps150_error.bin')
    RMSE_txt = floder_path+'\\RMSE.txt'
    text_file = open(RMSE_txt, 'w')
    print('RMSE: ', np.sum(error)/timesteps, file = text_file)
    text_file.close()

In [359]:
air_raw_reconsted = np.fromfile('./Data_set/AirTemp_raw_reconted_Timesteps150_x160_y320_NumOfBins128.bin', dtype=np.float32).reshape((150,160,320,128))
air_data = np.load('./Data_set/Air_temp_160_320_ensemble60_timestep150.npy')

In [12]:
path = 'C:\\Users\\User\\Desktop\\master thesis\\Experiment\\Tgmm_slic\\Airta\\160_320\\75'
floder_list = os.listdir(path)
raw_hist_data = air_raw_reconsted
timesteps = 150
cons_time = 75
num_of_bins = 128
for i in range (0,len(floder_list),13):
    print(floder_list[i])
    floder_path = os.path.join(path, floder_list[i])
    compute_file_error(floder_path, raw_hist_data, timesteps, cons_time, num_of_bins, 160, 320)

In [367]:
climate_interp = interpolated_hist(air_data, 150, 60, 30)
climate_interp_error = np.zeros(150)
for i in range(150):
    climate_interp_error[i] = compute_error(climate_interp[:,:,i], air_raw_reconsted[i])
climate_interp_error.astype('float32').tofile('climate_interp_sampletime30_error.bin')
print('error: ', sum(climate_interp_error)/150)

[  0  29  59  89 119 149]
storage:  14.149488 (MB)
error:  0.05571792989582556


In [120]:
path = 'C:\\Users\\User\\Desktop\\master thesis\\Experiment\\Tgmm_slic\\Nyx_density\\New_32CUBE\\200'
floder_list = os.listdir(path)
floder_path = os.path.join(path, floder_list[-1])
error_path = os.path.join(floder_path, os.listdir(floder_path)[-2])
error = np.fromfile(error_path, dtype = np.float32)

In [122]:
nyx_reconsted = quick_create_reconstruct(floder_path, 128, 0, 200,32,32,32)
nyx_reconst_data = nyx_reconsted.new_all_reconstruct(200)

In [374]:
nyx_raw_reconsted = np.fromfile('./Data_set/nyx_32cube_raw_reonted_200_32_32_32_128.bin', dtype=np.float32).reshape((200,32,32,32,128))
nyx_data = np.load('./Data_set/nyx_density_32cube_ensemble64_timestep200.npy')

In [375]:
interp = interpolated_hist(nyx_data, 200, 64, 10)
nyx_interp_error = np.zeros(200)
for i in range(200):
    nyx_interp_error[i] = compute_error(interp[:,:,:,i], nyx_raw_reconsted[i])
nyx_interp_error.astype('float32').tofile('Nyx_interp_10_error.bin')
print('error: ', sum(nyx_interp_error)/200)

[  0   9  19  29  39  49  59  69  79  89  99 109 119 129 139 149 159 169
 179 189 199]
storage:  17.638576 (MB)
error:  0.04127082979862939


In [158]:
path = 'C:\\Users\\User\\Desktop\\master thesis\\Experiment\\Tgmm_slic\\Nyx_density\\New_32CUBE\\50'
floder_list = os.listdir(path)
raw_hist_data = nyx_raw_reconsted
timesteps = 200
cons_time = 50
num_of_bins = 128
for i in range (0,len(floder_list),5):
    print(floder_list[i])
    floder_path = os.path.join(path, floder_list[i])
    compute_file_error(floder_path, raw_hist_data, timesteps, cons_time, num_of_bins, 32, 32, 32)

# for floder in floder_list:
#     print(floder)
#     floder_path = os.path.join(path, floder)
#     compute_file_error(floder_path, raw_hist_data, timesteps, cons_time, num_of_bins, 32, 32, 32)

Nyx_32Cube_NumOfEns64_Timesteps200_ConsTimes50_NumOfComps1_NonSLIC
Nyx_32Cube_NumOfEns64_Timesteps200_ConsTimes50_NumOfComps3_NonSLIC
Nyx_32Cube_NumOfEns64_Timesteps200_ConsTimes50_NumOfComps5_NonSLIC
Nyx_32Cube_NumOfEns64_Timesteps200_ConsTimes50_NumOfComps7_NonSLIC


In [155]:
path = 'C:\\Users\\User\\Desktop\\master thesis\\Experiment\\Tgmm_slic\\Nyx_density\\New_32CUBE\\100'
floder_list = os.listdir(path)
raw_hist_data = nyx_raw_reconsted
timesteps = 200
cons_time = 100
num_of_bins = 128
for i in range(1,len(floder_list),20):
    floder = floder_list[i]
    print(floder_list[i])
    floder_path = os.path.join(path, floder)
    compute_file_error(floder_path, raw_hist_data, timesteps, cons_time, num_of_bins, 32, 32, 32)

# for floder in floder_list:
#     print(floder)
#     floder_path = os.path.join(path, floder)
#     compute_file_error(floder_path, raw_hist_data, timesteps, cons_time, num_of_bins, 32, 32, 32)

Nyx_32Cube_NumOfEns64_Timesteps200_ConsTimes100_NumOfComps1_SLIC_64_100_12


In [343]:
redsea_data = np.load('./Data_set/RedSea_salt_25_20_120_ensemble50_timestep60.npy')
redsea_raw_reconsted = np.fromfile('./Data_set/RedSea_raw_reconted_60_25_20_120_128.bin', dtype=np.float32).reshape((60,25,20,120,128))

In [358]:
redsea_interp = interpolated_hist(redsea_data, 60, 50, 8)
redsea_interp_error = np.zeros(60)
for i in range(60):
    redsea_interp_error[i] = compute_error(redsea_interp[:,:,:,i], redsea_raw_reconsted[i])
redsea_interp_error.astype('float32').tofile('redsea_interp_sampletime8_error.bin')
print('error: ', sum(redsea_interp_error)/60)

error:  0.009344510949768766


In [339]:
path = 'C:\\Users\\User\\Desktop\\master thesis\\Experiment\\Tgmm_slic\\RedSea\\new\\30'
floder_list = os.listdir(path)
raw_hist_data = redsea_raw_reconsted
timesteps = 60
cons_time = 30
num_of_bins = 128
for i in range(8,len(floder_list)):
    floder = floder_list[i]
    print(floder_list[i])
    floder_path = os.path.join(path, floder)
    compute_file_error(floder_path, raw_hist_data, timesteps, cons_time, num_of_bins, 25, 20, 120)

Redsea_NumOfEns50_Timesteps60_ConsTimes30_NumOfComps5_SLIC_1024_50_3
Redsea_NumOfEns50_Timesteps60_ConsTimes30_NumOfComps5_SLIC_1024_50_5
Redsea_NumOfEns50_Timesteps60_ConsTimes30_NumOfComps5_SLIC_1024_50_7
Redsea_NumOfEns50_Timesteps60_ConsTimes30_NumOfComps5_SLIC_1024_50_9
Redsea_NumOfEns50_Timesteps60_ConsTimes30_NumOfComps7_SLIC_1024_50_3
Redsea_NumOfEns50_Timesteps60_ConsTimes30_NumOfComps7_SLIC_1024_50_5
Redsea_NumOfEns50_Timesteps60_ConsTimes30_NumOfComps7_SLIC_1024_50_7
Redsea_NumOfEns50_Timesteps60_ConsTimes30_NumOfComps7_SLIC_1024_50_9


In [342]:
path = 'C:\\Users\\User\\Desktop\\master thesis\\Experiment\\Tgmm_slic\\RedSea\\new\\20'
floder_list = os.listdir(path)
raw_hist_data = redsea_raw_reconsted
timesteps = 60
cons_time = 20
num_of_bins = 128
for i in range(0,8):
    floder = floder_list[i]
    print(floder_list[i])
    floder_path = os.path.join(path, floder)
    compute_file_error(floder_path, raw_hist_data, timesteps, cons_time, num_of_bins, 25, 20, 120)

Redsea_NumOfEns50_Timesteps60_ConsTimes20_NumOfComps1_SLIC_1024_50_3
Redsea_NumOfEns50_Timesteps60_ConsTimes20_NumOfComps1_SLIC_1024_50_5
Redsea_NumOfEns50_Timesteps60_ConsTimes20_NumOfComps1_SLIC_1024_50_7
Redsea_NumOfEns50_Timesteps60_ConsTimes20_NumOfComps1_SLIC_1024_50_9
Redsea_NumOfEns50_Timesteps60_ConsTimes20_NumOfComps3_SLIC_1024_50_3
Redsea_NumOfEns50_Timesteps60_ConsTimes20_NumOfComps3_SLIC_1024_50_5
Redsea_NumOfEns50_Timesteps60_ConsTimes20_NumOfComps3_SLIC_1024_50_7
Redsea_NumOfEns50_Timesteps60_ConsTimes20_NumOfComps3_SLIC_1024_50_9


In [33]:
air_data = np.load('Air_temp_180_320_ensemble60_timestep150.npy')
inter_ta_error = np.zeros(150)
for i in range(150):
    inter_ta_error[i] = compute_error(interpolated_hist(air_data, 150, 60, 10, i),air_raw_reconsted)
inter_ta_error.astype('float32').tofile('air_ta_interp_10_error.bin')
print('air ta interp 10 error: ', np.sum(inter_ta_error)/150)

FileNotFoundError: [Errno 2] No such file or directory: 'Air_temp_180_320_ensemble60_timestep150.npy'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
fig = plt.figure(figsize = (30,5))
vmin = np.min(air_raw_iso)
vmax = np.max(air_raw_iso)
norm = matplotlib.colors.Normalize(vmin = vmin, vmax = vmax)
plt.subplot(1,4,1)
rawplot = plt.imshow(air_raw_iso,cmap='coolwarm' , norm = norm, origin='lower')
plt.title('Raw')
# p1 = plt.colorbar(rawplot)

# plt.subplot(1,4,2)
# slic4_plot = plt.imshow(sz_air_iso, cmap='coolwarm', norm = norm, origin='lower')
# plt.title('sz')
# # p2 = plt.colorbar(slic4_plot)

# plt.subplot(1,4,3)
# slic8_plot = plt.imshow(inter_iso, cmap='coolwarm', norm = norm,origin='lower')
# plt.title('inter')
# p3 = plt.colorbar(slic8_plot)

# plt.subplot(1,4,4)
# slic64_plot = plt.imshow(airtemp_7_50_time_1_iso, cmap='coolwarm', norm = norm, origin='lower')
# plt.title('SLIC comp 12')
# # p3 = plt.colorbar(slic64_plot)

plt.show()

